(tmux_workflow)=
# Рабочий процесс с tmux

Сам по себе tmux - это просто еще один терминал с несколькими прибамбасами, которые позволяют нам отображать ... больше сеансов терминала.
Но tmux упрощает работу с программами, которые мы запускаем в этих сеансах, поэтому в этой главе будут рассмотрены некоторые распространенные и необычные конфигурации и команды, которые могут оказаться полезными в вашей повседневной работе.
Вы увидите несколько продвинутых способов управления вашими панелями и сеансами, как заставить tmux работать с выбранной вами оболочкой, как расширить команды tmux с помощью внешних скриптов и как создать привязки клавиш, которые выполняют несколько команд.
Давайте начнем с окон и панелей.

## Эффективная работа с панелями и окнами

На протяжении всей книги вы видели способы разделения сеансов tmux на панели и окна.
В этом разделе мы рассмотрим более продвинутые способы работы с ними.

### Превращение панели в окно
Панели отлично подходят для разделения рабочего пространства, но иногда вы можете захотеть превратить панель в полноценное окноа.
У tmux есть команда для этого.

В любой панели нажмите `PREFIX` `!` и tmux создаст новое окно на из вашей панели, удалив исходную панель.

### Превращение окна в панель
Иногда полезно объединить рабочее пространство.
Вы можете легко взять окно и превратить его в панель.
Для этого выполните команду `join-pane`.
Попробуйте. Создайте новый сеанс tmux с двумя окнами.

```
$ tmux new-session -s panes -n first -d
$ tmux new-window -t panes -n second
$ tmux attach -t panes
```

Теперь, чтобы переместить первое окно на панель второго окна, нажмите `PREFIX` `:` для входа в командный режим, и введите команду:

`join-pane -s panes:1`


Это означает «Взять окно 1 сеанса panes и присоединить его к текущему окну», поскольку мы не указали цель.
Когда вы «присоединяетесь» к панели, вы, по сути, перемещаете панель из одного сеанса в другой.
Вы указываете исходное окно и панель, а затем целевое окно и панель.
Если вы не указываете цель, текущее активное окно становится целью.

Вы также можете использовать эту технику для перемещения панелей.
Если бы ваше первое окно имело две панели, вы могли бы указать исходную панель, имея в виду, что вы устанавливаете базовые индексы окна и панели равными 1 вместо 0 в Главе 2, «Настройка tmux» таким образом

`join-pane -s panes:1.1`

Эта команда захватывает первую панель первого окна и соединяет ее с текущим окном.

Чтобы сделать еще один шаг, вы можете указать другой исходный сеанс, используя нотацию [имя_сеанса]:[окно].[панель] , и вы можете указать целевое окно, используя флаг `-t`, используя ту же нотацию.
Это позволяет вам перемещать панели из одного сеанса в другой.

### Развертывание и восстановление панелей

Иногда вам просто нужно, чтобы панель ненадолго стала полноэкранной, чтобы вы могли видеть ее содержимое или работать с ней более удобным образом.
Вы можете использовать команду `break-pane`.
Но тогда вам придется использовать `join-pane`, чтобы вернуть ее на свое прежнее место.
Оказыватся есть лучший способ.
Команда `resize-pane` принимает параметр `-Z` для масштабирования панели.
Лучше всего то, что он уже сопоставлен с `PREFIX` `z` , и повторное нажатие восстанавливает исходный размер панели.

### Запуск команд в панелях
В Главе 3 «Создание настраиваемых рабочих окружений в tmux» мы рассмотрели, как использовать команды терминала и `send-keys` для запуска программ в наших панелях, но мы можем выполнять команды автоматически при запуске окна или панели.

У нас есть два сервера, `burns` и `smithers`, на которых работают наш веб-сервер и сервер базы данных соответственно. Когда мы запускаем tmux, мы хотим подключиться к этим серверам, используя одно окно с двумя панелями.

Давайте напишем новый скрипт с именем server.sh создающий один сеанс, подключающийся к двум серверам:

```
$ tmux new-session -s servers -d "ssh deploy@burns"
$ tmux split-window -v "ssh dba@smithers"
$ tmux  attach -t servers
```

Когда мы создаем новый сеанс, мы можем передать команду, которую хотим выполнить, в качестве последнего аргумента.
В нашем случае мы запускаем новый сеанс и подключаемся к `burns` в первом окне, а затем отсоединяемся от сеанса.
Затем мы разделяем окно вертикально, а затем подключаемся к `smithers`.

Эта конфигурация имеет удобный побочный эффект: когда мы отключимя от наших удаленных серверов, панель или окно закроется.

### Открытие панели в текущем каталоге
Когда вы открываете новую панель, tmux помещает вас в каталог, в котором вы изначально запустили tmux.
Иногда это именно то, что вам нужно, но если вы перешли в другой каталог, вы можете захотеть вместо этого создать новую панель, которая откроется в этом же каталоге.

Вы можете использовать переменную `pane_current_path`, предоставляемую tmux, при создании новой панели.
Войдите в командный режим и выполните:

`split-window -v -c "#(pane_current_path)"`

Это разделит окно по горизонтали, и создаст новый сеанс терминала в том же рабочем каталоге, что и текущая панель или окно.
Вы можете добавить такое решение в свой файл конфигурации, чтобы работать было проще.
Вместо изменения существующих горячих клавиш для разбиения окон добавляйте новые, чтобы выбрать нужное поведение:

```
config/tmux.conf
# Split pane and retain the current directory of existing pane

bind _ split-window -v -c "#(pane_current_path)"
bind \ split-window -h -c "#{pane_current_path}"
```

Эти настройки позволяют нажимать `PREFIX` `_` для разделения окн по горизонтали, а `PREFIX` `/` - по вертикали.


### Одновременное выполнение команд в нескольких  панелях
Время от у вас может возникнуть необходимость выполнить одну и ту же команду в нескольких панелях.
Например, вам может понадобиться запустить один и тот же сценарий обновления на двух серверах.
Вы можете легко сделать это с помощью tmux.

Используя команду `set-window-option synchronize-panes on`, все, что вы вводите в одной панели, будет немедленно транслироваться на другие панели в текущем сеансе.
После того, как вы ввели команду, вы можете отключить режим сирхронизации используя `set-window-option synchronize-panes off`.
Чтобы упростить этот процесс, вы можете сопоставить это, например, с `PREFIX` `CTRL-s`:

```
config/tmux.conf

# Shortcut for synchronize-panes toggle
bind C-s set-window-option synchronize-panes
```

Без указания параметра `on` или `off`, команда `synchronize-panes` действует как переключатель.
Хоть это и не то, чем вы будете пользоваться очень часто, это удивительно удобно, когда вам это нужно.

## Управление сеансами

Когда вы освоитесь с tmux, вы можете обнаружить, что используете более одного сеанса tmux одновременно.
Например, вы можете запускать уникальные сеансы tmux для каждого приложения, над которым вы работаете так что вы можете держать среды изолированными.
Есть несколько замечательных функций tmux, позволяющих легко управлять сеансами.

### Переключение  между сессиями
Все сеансы tmux на одном компьютере размещаются на одном сервере.
Это означает, что вы можете легко переключаться между сеансами с одного клиента.

Давайте опробуем это.
Запустите два отдельных сеанса tmux, один с именем «editor», который запускает Vim, а другой, выполняющиq команду `top`, назовем «processes»:
```
$ tmux new -s editor -d vim
$ tmux new -s processes -d top
```

Подключитесь к сеансу `editor`

`tmux attach -t editor`

а затем нажмите `PREFIX` `(` для перехода к предыдущему сеансу и `PREFIX` `(` для перехода к следующему сеансу.

Вы также можете использовать `PREFIX` `s` для отображения списка сеансов, чтобы можно было быстро переключаться между ними

<!-- TODO
![](./img/-096-022.png)
-->

Еще можно использовать клавиши `j` и `k` для перемещения вверх и вниз, если вы настроили tmux для использования Vim-подобного управления курсором, или можете нажать `ПРОБЕЛ`, чтобы развернуть сеанс, когда вам нужно перейти к определенному окну или панели.

Для этого вы можете добавить настройки горячих клавиш в файле .tmux.conf, привязав клавиши к команде `switch-client`.
Конфигурация по умолчанию выглядит так:

```
config/.tmux.conf

# Shortcut for moving between sessions
bind ( switch-client -p
bind ) switch-client -n
```

Если вы настроили несколько рабочих окружений, это чрезвычайно эффективный способ перемещения между ними, без отсоединения и повторного присоединения.

### Перемещение окон между сеансами
Вы можете перемещать окно из одного сеанса в другой.
Это удобно, если вы запустили процесс в одной окружении и хотите переместить его или хотите объединить свои рабочие пространства.

Команда ` move-window` связана с `PREFIX` `.` (точка), так что вы может перейти к окну, которое вы хотите переместить, нажать комбинацию клавиш, а затем ввести имя целевого сеанса.

Чтобы попробовать это, создайте два сеанса с именами «editor» и «processes», запустив в них `vim` и `top` соответственно:

```
$ tmux new -s editor -d vim
$ tmux new -s processes -d top
```

Переместим окно из сеанса «processes» в сеанс «editor».

Во-первых, подключитесь к сеансу «processes» следующим образом:

`$ tmux attach -t processes`

Затем нажмите `PREFIX` `.` и введите `processes` в появившейся командной строке.

Это удалит единственное окно в сеансе «processes», вызывая его закрытие. Если подключиться к сеансу «editor», вы увидите оба окна.

Для этого вы также можете использовать команды оболочки, так что вам даже не нужно открывать сеансы.
Воспользуйте для этого `move-window`:

`$ tmux move-window -s processes:1 -t editor`


Это переместит первое окно сеанса «processes» в сеанс «editor».

### Создание или присоединение к существующим сеансам
До сих пор мы всегда использовали подход создания новых сеансов tmux всякий раз, когда хотели что-то сделать.
Однако на самом деле мы можем определить, существует ли сеанс tmux, и подключиться к нему, если он есть.

Команда `has-session` возвращает логическое значение, которое мы можем использовать в скриптах.
Это означает, что мы можем сделать что-то вроде этого в сценарии Bash:

```
if ! tmux has-session -t development; then
  exec tmux new-session -s development -d
  # other setup commands before attaching....
fi
  exec tmux attach -t development
```

Если вы измените сценарий так, чтобы он принимал аргумент, вы можете использовать его для создания одного сценария, который можно использовать для подключиться или создания любых сеансов tmux.

## tmux и ваша операционная система

По мере того, как tmux становится частью вашего рабочего процесса, вы можете захотеть более тесно интегрировать его с вашей операционной системой.
В этом разделе вы узнаете, как заставить tmux и вашу систему работать вместе.

### Использование другой оболочки
В этой книге мы использовали оболочку Bash, но если вы поклонник zsh, вы все равно можете получить все преимущества tmux.
Просто явно установите оболочку по умолчанию в .tmux.conf следующим образом:

```
config/tmux.conf

# Using a Different Shell zsh
set -g default-shell /bin/zsh
```

Поскольку tmux — это просто терминальный мультиплексор, а не собственная оболочка, вы просто указываете, что именно запускать
при его старте.

### Запуск tmux по умолчанию
Вы можете настроить свою систему на автоматический запуск tmux при открытии терминала.
И используя то, что вы знаете об именах сеансов, вы можете создать новый сеанс, если таковой не существует, или прикрепиться к тому, который существует.

Когда tmux запущен, он устанавливает для переменной `TERM` значение «screen» или значение `default-terminal` из файла конфигурации.
Вы можете использовать это значение в файле .bashrc (или .bash_profile в macOS), чтобы определить, находитесь ли вы в настоящее время в сеансе tmux.
Вы снова установили свой терминал tmux на «screen-256color» в
Вы настроили свой терминал tmux на «screen-256color» еще в главе 2 «Настройка tmux», и можете спользовать это для определения того, действительно ли запущен tmux.

Например, вы можете добавить эти строки в конец вашего файла `.bashrc`:

```
if [[ "$TERM" != "screen-256color"]]
then
  tmux attach-session -t "$USER" || tmux new-session -s"$ USER" fi
```

Здесь первоначально проверяется, что вы еще не находитесь в сеансе tmux. Если это так, производится попытка подключиться к
сеансу с именем `$USER`, которое является именем вашего пользователя.
Вы можете заменить это любым значением, которое хотите, но использование имени пользователя помогает избежать конфликтов.

Если сеанс не существует, tmux выдаст ошибку, которую сценарий оболочки может интерпретировать как ложное значение.
Затем выполняется правая часть выражения, которая создает новый сеанс с вашим именем пользователя в качестве имени сеанса.
На этом сценарий заканчивается.

Когда tmux запустится снова, он снова прочитает наш файл .bashrc или .bash_profile, но на этот раз он увидит, что мы находимся в сеансе tmux, пропустит этот фрагмент кода и выполнит остальные команды в нашем файле конфигурации, гарантируя, что все наши переменные среды являются настроенными для нас.

Теперь каждый раз, когда вы открываете новый терминал, вы будете в сеансе tmux. Однако будьте осторожны, так как каждый раз вы открываете новый сеанс терминала на своем компьютере, он будет присоединен к тому же сеансу.
Выход из tmux в одном терминале выйдет из tmux во всех из них.

### Хранение специфических конфигурации
В главе 4 «Работа с текстом и буферами» вы узнали, как заставить tmux работать с системными буферами обмена Linux и OS X и, и это потребовало добавления некоторых специфичных параметров в ваш файл .tmux.conf.
Но если вы хотите, чтобы ваша конфигурация работала в обеих операционных системах, вы столкнетесь с некоторыми конфликтами.

Решение состоит в том, чтобы переместить конфигурацию, зависящую от вашей ОС, в отдельный файл, а затем заставить tmux загрузить ее с помощью команд `if-shell` и `source`.
Попробуйте  это.
Создайте новый файл с именем .tmux.mac.conf в своем домашнем каталоге:

```
$ touch ~/.tmux.mac.conf`
```

В этот файл поместите весь код интеграции tmux c буферjv обмена Mac:

```
config/tmux.mac.conf

# Setting the namespace
set -g default-command "reattach-to-user-namespace -l /bin/bash"

# Prefix C-c copy buffer to system clipboard
bind C-c run " tmux save-buffer - | reattach-to-user-namespace pbcopy"

# Prefix C-v paste system clipboard into tmux
bind C-v run \
"tmux set-buffer \"$ (reattach-to-user-namespace pbpaste)\"; tmux paste-buffer"

# use y in visual mode to copy text to system clipboard
bind -t vi-copy y copy-pipe ​"reattach-to-user-namespace pbcopy"
```

Затем откройте .tmux.conf и удалите все строки, относящиеся к OS X, если вы их добавили.
Затем добавьте в конец файла следующие строки:

```
config/tmux.conf

# Load mac-specific settings
if-shell "uname | grep -q Darwin" "source-file ~/.tmux.mac.conf"
```

Команда `if-shell` запускает команду оболочки, и в случае успеха выполняет шаг.
В этом случае мы указываем tmux запустить команду `uname` и использовать `grep`, чтобы увидеть, содержит ли она слово «Darwin».
Если это так, то можно с уверенностью сказать, что мы на Mac, поэтому мы загружаем файл конфигурации.
Вы можете использовать аналогичный подход для загрузки дополнительных параметров конфигурации в зависимости от других условий.
Например, вы возможно захотите поделиться своим главным файлом .tmux.conf со всем миром на GitHub, но, возможно, вы захотите сохранить некоторые свои собственные секретные данные в секрете. Поэтому переместите все эти настройки в .tmux.private и добавьте его загрузку в свой файл .tmux.conf:

```
config/tmux.conf

# Load private settings if they exist
if-shell "[-f ~/.tmux.private]" "source ~/.tmux.private"
```
Это позволит загрузить файл только в том случае, если он существует.

### Запись выходных данных программы в журнал

Иногда полезно иметь возможность записывать вывод сеанса терминала в журнал.
Вы уже узнали как для этого использовать `capture-pane` and `save-buffer`, но tmux может фактически записывать действия на панели прямо в  текстовый файл с помощью команды `pipe-pane`.
Это похоже на команду `script`, доступную во многих оболочках, за исключением что с помощью `pipe-pane` вы можете включать и выключать данный режим, и вы можете запускать его после того, как программа уже выполнилась.

Для активиции данного режима, войдите в командный режим и введите `pipe-pane -o "cat >> mylog.txt`.

Вы можете использовать флаг `-o` для переключения вывода, что означает, что если вы снова отправите такую команду, вы можете отключить логгирование.
Чтобы упростить выполнение этой команды, назначьте сочетание клавиш для нею в свой файл конфигурации.

```
config/tmux.conf

# Log output to a text file on demand
bind P pipe-pane -o "cat >>~/#W.log" \; display "Toggled logging to ~/#W.log"
```
Теперь вы можете нажать `PREFIX` `P`, чтобы включть/выключить ведение журнала.

Благодаря команде `display` (сокращение от `display-message`),
вы увидите имя файла журнала, отображаемое в строке состояния. Команда `display` имеет доступ к тем же переменным, что и строка состояния, которые указзаны в Таблице 1 Переменные строки состояния.

###  Добавление срока службы батареи в строку состояния

Если вы используете tmux на ноутбуке, вы можете захотеть показать оставшееся время работы от батареи в строки состояния состояния, особенно если вы запускаете свой терминал в полноэкранном режиме.
Этого очень легко добиться переменной  #(shell-command).
Давайте добавим состояние батареи в наш файл конфигурации. Возьмем сценарий оболочки, который может получить оставшиеся заряд батареи и отображать его на экране.
Поместим этот файл с именем battery в нашей домашней папке и сообщим tmux, чтобы он запускал его.

Сначала скачайте файл:

```
wget --no-check-certificate \
    https://raw.github.com/richo/battery/master/bin/battery
```


Вы также можете найти скрипт `battery` в загрузках исходного кода книги.

Теперь сделайте его исполняемым, чтобы tmux мог его использовать:

`$ chmod +x ~/battery`

Проверьте как это будет выглядить
`$ ~/battery Discharging`

Если вы запускаете это на ноутбуке не подключенного к электросети, вы увидите, сколько процентов осталось в батарее.

Мы можем заставить tmux отображать вывод любой программы командной строки в строке состояния, используя #(<command>).

Итак, чтобы отображать батарею перед часами, измените строку состояния справа в .tmux.conf на это:

```
config/tmux.conf

# Status line right side - 50% | 31 Oct 13:37​
set -g status-right "#(~/battery Discharging) | #[fg=cyan]%d %b %R"
```

Теперь, когда вы перезагрузите файл .tmux.conf, появится индикатор состояния батареи.

<!-- TODO
![](./img/-101-023.png)
-->

Чтобы получить состояние батареи, когда она заряжается, вам нужно выполнить команду

`$ ~/battery Charging`

и вставьте это в строку состояния.
Я оставлю это на ваше усмотрение.

Вы можете использовать этот подход для дальнейшей настройки строки состояния.
Вам просто нужно написать свой собственный
скрипт, который возвращает значение, которое вы хотите отобразить, а затем поместите его в строку состояния.

### Полная интеграция с Vim
Текстовый редактор Vim довольно хорошо работает с tmux, но разработчик Мислав Марохнич разработал решение, которое позволяет легко перемещаться между панелями tmux и Vim.
Чтобы это работало, вам нужно установить плагин `vim-tmux-navigator` Криса Туми для Vim[16] и добавить несколько сочетаний клавиш в ваш файл .tmux.conf.

Эта настройка создаст следующие сочетания клавиш:

`CTRL-j` двигаться вверх

`CTRL-k` двигаться вниз

`CTRL-h` двигаться влево

`CTRL-l` двигаться вправо

Если вы находитесь в tmux и переходите на Vim, плагин Vim берет на себя управление.
Если вы находитесь в Vim и переходите в tmux, тогда tmux берет на себя управление.
Вместо того, чтобы изучать два набора команд для навигации, вы
просто будете иметь один.

Чтобы настроить это, установите плагин Vim с помощью Vundle, добавив его в свой файл .vimrc:

`Plugin 'christoomey/vim-tmux-navigator'`

Затем сохраните файл .vimrc и запустите

` :PluginInstall`

в Vim для установки плагина.

Затем в .tmux.conf добавьте следующие строки:

```
config/tmux.conf

# Integrating Seamlessly with Vim
is_vim="ps -o state= -o comm= -t '#{pane_tty}' \
  | grep -iqE '^[^TXZ ]+ +(\\S+\\/)?g?(view|n?vim?x?)(diff)?$'"
bind-key -n C-h if-shell "$is_vim" "send-keys C-h" "select-pane -L"
bind-key -n C-j if-shell "$is_vim" "send-keys C-j" "select-pane -D"
bind-key -n C-k if-shell "$is_vim" "send-keys C-k" "select-pane -U"
bind-key -n C-l if-shell "$is_vim" "send-keys C-l" "select-pane -R"
bind-key -n C-\ if-shell "$is_vim" "send-keys C-\\" "select-pane -l"

bind C-l send-keys 'C-l'
```

`CTRL-l` — это сочетание клавиш, используемое библиотекой `readline` во многих оболочках для очистки экрана.
    Последняя строка этой конфигурации устанавливает `PREFIX` `CTRL-l` вместо этой команды.

## Расширение tmux с помощью плагинов

До сих пор мы вносили изменения непосредственно в файл конфигурации tmux.
Хотя это работает, сделать так что-то более сложное может быть немного неудобно.
Бруно Сутик разработал решение этой проблемы под названием TPM, менеджер плагинов tmux (tmux plugin manager).
С тех пор все больше и больше людей объединяются для создания плагинов для расширения tmux.
Давайте воспользуемся TPM для установки невероятно полезного плагина `tmux-resurrect` [17], который может восстанавливать сеансы tmux даже после перезагрузки!
Чтобы настроить его, сначала клонируйте репозиторий в папку с именем ~/.tmux/plugins/tpm:

`$ git clone https://github.com/tmux-plugins/tpm ~/.tmux/plugins/tpm`

Затем добавьте эти строки в ваш файл .tmux.conf:

```
config/tmux.conf
set -g @plugin 'tmux-plugins/tpm'
set -g @plugin 'tmux-plugins/tmux-resurrect'
run '~/.tmux/plugins/tpm/tpm'
```

Сначала мы указываем сам TPM, а затем подключаемый модуль tmux-resurrect.
Далее мы загружаем TPM, чтобы он мог загружать другие плагины.
Сохраните этот файл и перезагрузите конфигурацию.
Затем нажмите `PREFIX` `I`, чтобы установить плагин.
Вы увидите следующий вывод:

```
Already installed "tpm"

Installing "tmux-resurrect"
  "tmux-resurrect" download success

TMUX environment reloaded.

Done, press ENT ER to continue.
```

Теперь протестируйте программу `tmux-resurrect`.
Откройте еще пару панелей, а затем нажмите `PREFIX` `CTRL-s`, чтобы сохранить состояние сеанса tmux.
Затем закройте все панели и выйдите из tmux.
Наконец, перезагрузите tmux и нажмите `PREFIX` `CTRL-r` для восстановления сохраненного сеанса.
Все ваши окна вернутся!
Посетите список плагинов tmux[18] и найдите тот, который хотите установить.
Вы найдете один для тестомера, который мы настроили, другой для буфера обмена конкретной операционной системы и даже один с разумными параметрами конфигурации.

## Что дальше?

Теперь, когда вы знаете основы и имеете некоторый опыт работы с различными конфигурациями, вы можете сделать с tmux гораздо больше.

Руководство по tmux, к которому вы можете получить доступ с вашего терминала

`$ man tmux`

содержит полный список опций конфигурации и доступных команд.

И не забывайте, что сам tmux быстро развивается.
В следующей версии появятся новые параметры конфигурации, что даст вам еще больше гибкости.

Когда вы интегрируете tmux в свой рабочий процесс, вы можете открыть для себя другие методы, на которые вы начнете полагаться.
Например, вы можете использовать tmux и текстовый редактор на удаленном сервере для создания невероятно эффективной
среды разработки, которую вы можете использовать для совместной работы с другим разработчиком.
Вы даже можете использовать irssi ((клиент IRC на базе терминала) и Alpine (приложение электронной почты на основе терминала) в сеансах tmux либо рядом с вашим текстовым редактором на панели или в фоновых окнах.
Вы даже вы можете отключиться от сеанса и вернуться к нему позже, и вся ваша среда будет готова к работе.

Продолжайте работать с tmux, и, прежде чем вы это поймете, он станет неотъемлемой частью вашего рабочего процесса.

## Для дальнейшего использования



 Команда             | Описание
:--------------------|:----------
`PREFIX` `!` | Преобразует текущую панель в новое окно.
`join-pane -s [session]:[window].[pane]` | Преобразует окно или панель указанного сеанса в панель в текущем окне.
`join-pane -s [session]:[window].[pane] -t [other
session]` | Преобразует окно или панель указанного сеанса в панель целевого сеанса.
`PREFIX` `z` | Увеличивает масштаб текущей панели, делая ее полноэкранной. Повторное нажатие на нее восстанавливает первоначальный размер панели.
`tmux new-session "[command]"` | Запускает tmux и выполняет команду. Когда команда завершается, сеанс tmux закрывается.
`split-pane "[command]"` | Разбивает текущее окно и выполняет указанную команду в новой панели. Когда команда завершится, панель закроется.
`split-window -c "#{pane_current_path}"` | Разбивает панель и устанавливает рабочий каталог новой панели в текущий рабочий каталог сфокусированной панели.
`set-window-option synchronize-panes` | Переключает синхронизацию панелей, при которой нажатия клавиш выполняются для всех панелей одновременно, а не только для текущей панели.
`PREFIX` `(` | Переходит к следующему сеансу tmux.
`PREFIX` `)` | Переходит к предыдущему сеансу tmux.
`PREFIX` `s` | Показывает список выбора сеанса.
`move-window -s [source session]:[window] -t [target session]` | Перемещает окно из одного сеанса в другой. Также доступно через `PREFIX` `.', за которым следует имя целевого сеанса.
`set -g default-shell [shell]` | Задает оболочку по умолчанию, которую tmux использует при создании новых окон.
`set -g default-command [command]` | Задает команду по умолчанию, которую tmux использует при создании новых окон. По умолчанию пусто.
`if-shell "[condition]" "[command]"` | Выполняет заданную команду, если условие оценивается как `true`.
`pipe-pane -o "cat >>~/#W.log"` | Записывает текущую панель в текстовый файл.


#### Ссылки

16. https://github.com/christoomey/vim-tmux-navigator
17. https://github.com/tmux-plugins/tmux-resurrect
18. https://github.com/tmux-plugins